In [80]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [5]:
www = "https://www.europarl.europa.eu/meps/en/full-list/all"
r = requests.get(www)
soup = bs(r.text)

In [ ]:
links = [x.get("href") for x in soup.select(".erpl_member-list-item-content")]

In [117]:
links[:10]

['http://www.europarl.europa.eu/meps/en/197490',
 'http://www.europarl.europa.eu/meps/en/189525',
 'http://www.europarl.europa.eu/meps/en/124831',
 'http://www.europarl.europa.eu/meps/en/197826',
 'http://www.europarl.europa.eu/meps/en/197403',
 'http://www.europarl.europa.eu/meps/en/198096',
 'http://www.europarl.europa.eu/meps/en/125045',
 'http://www.europarl.europa.eu/meps/en/204335',
 'http://www.europarl.europa.eu/meps/en/197836',
 'http://www.europarl.europa.eu/meps/en/96750']

In [137]:
def scrape_page(www):
    
    print(www)
    
    r = requests.get(www)
    soup = bs(r.text)
    
    info = {}
    
    info["website"] = www
    
    mapping = {"name" : ".erpl_title-h1",
              "eu_party" : "h3.erpl_title-h3",
              "country_party" : "div.erpl_title-h3",
              "date_of_birth" : "#birthDate"}

    for key, value in mapping.items():
        
        try:
            info[key] = soup.select(value)[0].text.strip()
        except IndexError:
            info[key]  = ""
        
    info["parl_terms"] = [x.text for x in soup.find(text="History of parliamentary service").parent.parent.parent.select("span.t-x")[1:]]
    
    return info

In [ ]:
data = []

for member in links:
    m = scrape_page(member)
    data.append(m)

In [145]:
df = pd.DataFrame(data)

In [146]:
df

,name,eu_party,country_party,date_of_birth,parl_terms
0,Magdalena ADAMOWICZ,Group of the European People's Party (Christia...,Poland\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - In...,,[9th parliamentary term]
1,Asim ADEMOV,Group of the European People's Party (Christia...,Bulgaria\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - ...,03-12-1968,"[9th parliamentary term, 8th parliamentary term]"
2,Isabella ADINOLFI,Non-attached Members,Italy\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - Mov...,,"[9th parliamentary term, 8th parliamentary term]"
3,Matteo ADINOLFI,Identity and Democracy Group,Italy\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - Leg...,24-12-1963,[9th parliamentary term]
4,Alex AGIUS SALIBA,Group of the Progressive Alliance of Socialist...,Malta\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - Par...,31-01-1988,[9th parliamentary term]
...,...,...,...,...,...
699,Juan Ignacio ZOIDO ÁLVAREZ,Group of the European People's Party (Christia...,Spain\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - Par...,21-01-1957,[9th parliamentary term]
700,Carlos ZORRINHO,Group of the Progressive Alliance of Socialist...,Portugal\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - ...,28-05-1959,"[9th parliamentary term, 8th parliamentary term]"
701,Željana ZOVKO,Group of the European People's Party (Christia...,Croatia\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - H...,,"[9th parliamentary term, 8th parliamentary term]"
702,Marco ZULLO,Non-attached Members,Italy\r\n\t\t\t\t\t\t \r\n\t\t\t\t\t\t\t - Mov...,29-10-1978,"[9th parliamentary term, 8th parliamentary term]"


In [154]:
df["country"] = df["country_party"].apply(lambda x: x.split("-")[0].strip())

In [157]:
df["original_party"] = df["country_party"].apply(lambda x: x.split("-")[1].split("(")[0].strip())
#df[df["date_of_birth"] == ''].to_csv("missing_date.csv")

In [101]:
#df["parl_terms"] = df["parl_terms"].apply(lambda x: "|".join(x))

In [158]:
df["terms_in_parliament"] =  df["parl_terms"].apply(lambda x: len(x))

In [159]:
df.to_csv("data.csv")

In [119]:
#df["country"] = df["party_country"].apply(lambda x: x.split("("))

In [121]:
df["terms_in_parliament"].value_counts()

1    415
2    156
3     76
4     35
5     15
6      6
7      1
Name: terms_in_parliament, dtype: int64

In [120]:
df.sort_values("terms_in_parliament",ascending=False).head(20)

,name,party_country,date_of_birth,parl_terms,terms_in_parliament,country
358,Constanze KREHL,Group of the Progressive Alliance of Socialist...,14-10-1956,"[9th parliamentary term, 8th parliamentary ter...",7,[Group of the Progressive Alliance of Socialis...
388,Peter LIESE,Group of the European People's Party (Christia...,20-05-1965,"[9th parliamentary term, 8th parliamentary ter...",6,"[Group of the European People's Party , Christ..."
221,Michael GAHLER,Group of the European People's Party (Christia...,22-04-1960,"[9th parliamentary term, 8th parliamentary ter...",6,"[Group of the European People's Party , Christ..."
233,Evelyne GEBHARDT,Group of the Progressive Alliance of Socialist...,19-01-1954,"[9th parliamentary term, 8th parliamentary ter...",6,[Group of the Progressive Alliance of Socialis...
197,Markus FERBER,Group of the European People's Party (Christia...,15-01-1965,"[9th parliamentary term, 8th parliamentary ter...",6,"[Group of the European People's Party , Christ..."
247,Charles GOERENS,Renew Europe Group,06-02-1952,"[9th parliamentary term, 8th parliamentary ter...",6,[Renew Europe Group]
675,Rainer WIELAND,Group of the European People's Party (Christia...,19-02-1957,"[9th parliamentary term, 8th parliamentary ter...",6,"[Group of the European People's Party , Christ..."
387,Bogusław LIBERADZKI,Group of the Progressive Alliance of Socialist...,12-09-1948,"[9th parliamentary term, 8th parliamentary ter...",5,[Group of the Progressive Alliance of Socialis...
385,Janusz LEWANDOWSKI,Group of the European People's Party (Christia...,13-06-1951,"[9th parliamentary term, 8th parliamentary ter...",5,"[Group of the European People's Party , Christ..."
374,Bernd LANGE,Group of the Progressive Alliance of Socialist...,14-11-1955,"[9th parliamentary term, 8th parliamentary ter...",5,[Group of the Progressive Alliance of Socialis...
